# 面向对象的 JavaScript

## prototype

为了解决从原型对象生成实例的问题，JavaScript 提供了一个构造函数（Constructor）模式。

所谓"构造函数"，其实就是一个普通函数，但是内部使用了 `this` 变量。对构造函数使用 `new` 运算符，就能生成实例，并且 `this` 变量会绑定在实例对象上。如下：

In [1]:
// DOG 构造函数，表示狗对象的原型。
function Dog(name){
  this.name = name;
};

const dogA = new Dog('大毛');
console.log(dogA.name); // 大毛
console.log(dogA.constructor === Dog);  // true
console.log(dogA instanceof Dog); // true

大毛
true
true


用构造函数生成实例对象，有一个缺点，那就是无法共享属性和方法，如下：

In [2]:
function Dog(name){
  this.name = name;
  this.species = '犬科';
};

const dogB = new Dog('大毛');
const dogC = new Dog('二毛');

dogA.species = '猫科';
console.log(dogB.species); // 显示"犬科"，不受 dogA 的影响
console.log(dogB.species === dogA.species);  // false

犬科
false


要解决这个问题，可以为构造函数设置一个 `prototype` 属性，这个属性包含一个对象（以下简称"prototype对象"），所有实例对象需要共享的属性和方法，都放在这个对象里面；那些不需要共享的属性和方法，就放在构造函数里面。

实例对象一旦创建，将自动引用 `prototype` 对象的属性和方法。也就是说，实例对象的属性和方法，分成两种，一种是本地的，另一种是引用的。如下：

In [4]:
function Dog(name){
  this.name = name;
};

Dog.prototype = { species : '犬科' };

const dogD = new Dog('大毛');
const dogE = new Dog('二毛');

console.log(dogD.species); // 犬科
console.log(dogE.species); // 犬科

Dog.prototype.species = '猫科';

console.log(dogD.species); // 猫科
console.log(dogE.species); // 猫科
console.log(dogD.species === dogE.species)  // true
console.log(Dog.prototype.isPrototypeOf(dogD));  // true
console.log(dogD.hasOwnProperty("name"));  // true
console.log(dogD.hasOwnProperty("species"));  // false
console.log("name" in dogD);  // true
console.log("species" in dogD);  // true

犬科
犬科
猫科
猫科
true
true
true
false
true
true


由于所有的实例对象共享同一个 `prototype` 对象，那么从外界看起来，`prototype` 对象就好像是实例对象的原型，而实例对象则好像"继承"了 `prototype` 对象一样。这就是 Javascript 继承机制的设计思想。

## 参考

- [Javascript继承机制的设计思想](http://www.ruanyifeng.com/blog/2011/06/designing_ideas_of_inheritance_mechanism_in_javascript.html)
- [Javascript 面向对象编程（一）：封装](http://www.ruanyifeng.com/blog/2010/05/object-oriented_javascript_encapsulation.html)
- [Javascript面向对象编程（二）：构造函数的继承](http://www.ruanyifeng.com/blog/2010/05/object-oriented_javascript_inheritance.html)
- [Javascript面向对象编程（三）：非构造函数的继承](http://www.ruanyifeng.com/blog/2010/05/object-oriented_javascript_inheritance_continued.html)